# Text Generation Model - Production Kaggle Training
## 10M Parameter GPT-2 Style Model with Smart Checkpointing

**Features:**
- Train from scratch with proper error handling
- Automatic checkpoint management (keeps 4 most recent)
- Kaggle-optimized memory usage
- Resume training capability
- Progress tracking and logging

## 1. Environment Setup

In [ ]:
# Check environment
import sys
print(f"Python version: {sys.version}")
print(f"Running on Kaggle: {'/kaggle/working' in sys.path or 'KAGGLE_KERNEL_RUN_TYPE' in __import__('os').environ}")

In [ ]:
# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

## 2. Install and Import Dependencies

In [ ]:
# Kaggle has most packages pre-installed, but let's ensure versions are compatible
import subprocess
import sys

def install_package(package):
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])
        return True
    except:
        return False

# Only install if needed
try:
    import transformers
    import datasets
    print("✓ Core packages already installed")
except ImportError:
    print("Installing missing packages...")
    install_package('transformers>=4.30.0')
    install_package('datasets>=2.12.0')
    print("✓ Packages installed")

In [ ]:
# Import all required libraries
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import GPT2Config, GPT2LMHeadModel, GPT2Tokenizer
from transformers import get_linear_schedule_with_warmup
from datasets import load_dataset
import json
import glob
from pathlib import Path
from datetime import datetime
from tqdm.auto import tqdm
import gc
import math

print("✓ All imports successful")
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"Datasets version: {datasets.__version__}")

## 3. GPU Check and Configuration

In [ ]:
# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n{'='*60}")
print(f"Device: {device}")
print(f"{'='*60}")

if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"cuDNN Version: {torch.backends.cudnn.version()}")
    
    # Enable TF32 for better performance on Ampere GPUs
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    print("✓ TF32 enabled for faster training")
else:
    print("⚠️ WARNING: No GPU detected! Training will be extremely slow.")
    print("Please enable GPU in Kaggle notebook settings.")

print(f"{'='*60}")

## 4. Configuration

In [ ]:
# Training configuration
CONFIG = {
    # Model architecture
    'vocab_size': 50257,
    'n_positions': 512,
    'n_embd': 256,
    'n_layer': 8,
    'n_head': 8,
    'n_inner': 1024,
    
    # Training hyperparameters
    'batch_size': 8,
    'gradient_accumulation_steps': 8,  # Effective batch size = 64
    'learning_rate': 5e-4,
    'weight_decay': 0.01,
    'max_grad_norm': 1.0,
    'epochs': 3,
    'warmup_steps': 500,
    'max_length': 512,
    
    # Checkpointing
    'save_steps': 1000,
    'eval_steps': 500,
    'max_checkpoints': 4,  # Keep only 4 most recent checkpoints
    'checkpoint_dir': '/kaggle/working/checkpoints',
    
    # Dataset
    'dataset_name': 'wikitext',
    'dataset_config': 'wikitext-103-v1',
    
    # Resume training
    'resume_from_checkpoint': None,  # Set to checkpoint path to resume
}

# Create checkpoint directory
os.makedirs(CONFIG['checkpoint_dir'], exist_ok=True)

print("\n" + "="*60)
print("TRAINING CONFIGURATION")
print("="*60)
print(f"Model Parameters: ~10M")
print(f"Batch Size: {CONFIG['batch_size']}")
print(f"Gradient Accumulation: {CONFIG['gradient_accumulation_steps']}")
print(f"Effective Batch Size: {CONFIG['batch_size'] * CONFIG['gradient_accumulation_steps']}")
print(f"Learning Rate: {CONFIG['learning_rate']}")
print(f"Epochs: {CONFIG['epochs']}")
print(f"Max Checkpoints: {CONFIG['max_checkpoints']}")
print("="*60)

## 5. Checkpoint Management Functions

In [ ]:
def get_checkpoint_list(checkpoint_dir):
    """Get sorted list of checkpoint files"""
    checkpoints = glob.glob(os.path.join(checkpoint_dir, 'checkpoint_*.pt'))
    # Sort by modification time (newest first)
    checkpoints.sort(key=os.path.getmtime, reverse=True)
    return checkpoints

def cleanup_old_checkpoints(checkpoint_dir, max_keep=4):
    """Keep only the most recent N checkpoints"""
    checkpoints = get_checkpoint_list(checkpoint_dir)
    
    if len(checkpoints) > max_keep:
        to_delete = checkpoints[max_keep:]
        for ckpt in to_delete:
            try:
                os.remove(ckpt)
                print(f"  Deleted old checkpoint: {os.path.basename(ckpt)}")
            except Exception as e:
                print(f"  Warning: Could not delete {ckpt}: {e}")

def save_checkpoint(filepath, model, optimizer, scheduler, epoch, step, train_loss, val_loss=None, config=None):
    """Save training checkpoint"""
    try:
        checkpoint = {
            'epoch': epoch,
            'step': step,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
            'config': config,
            'timestamp': datetime.now().isoformat(),
        }
        
        torch.save(checkpoint, filepath)
        return True
    except Exception as e:
        print(f"Error saving checkpoint: {e}")
        return False

def load_checkpoint(filepath, model, optimizer=None, scheduler=None):
    """Load training checkpoint"""
    try:
        checkpoint = torch.load(filepath, map_location=device)
        
        model.load_state_dict(checkpoint['model_state_dict'])
        
        if optimizer is not None and 'optimizer_state_dict' in checkpoint:
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        
        if scheduler is not None and 'scheduler_state_dict' in checkpoint:
            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        
        return {
            'epoch': checkpoint.get('epoch', 0),
            'step': checkpoint.get('step', 0),
            'train_loss': checkpoint.get('train_loss', None),
            'val_loss': checkpoint.get('val_loss', None),
        }
    except Exception as e:
        print(f"Error loading checkpoint: {e}")
        return None

print("✓ Checkpoint management functions defined")

## 6. Model Initialization

In [ ]:
# Create model configuration
model_config = GPT2Config(
    vocab_size=CONFIG['vocab_size'],
    n_positions=CONFIG['n_positions'],
    n_embd=CONFIG['n_embd'],
    n_layer=CONFIG['n_layer'],
    n_head=CONFIG['n_head'],
    n_inner=CONFIG['n_inner'],
    activation_function='gelu_new',
    resid_pdrop=0.1,
    embd_pdrop=0.1,
    attn_pdrop=0.1,
)

print("\n" + "="*60)
print("MODEL ARCHITECTURE")
print("="*60)
print(f"Vocabulary Size: {model_config.vocab_size:,}")
print(f"Max Sequence Length: {model_config.n_positions}")
print(f"Embedding Dimension: {model_config.n_embd}")
print(f"Number of Layers: {model_config.n_layer}")
print(f"Number of Attention Heads: {model_config.n_head}")
print(f"FFN Inner Dimension: {model_config.n_inner}")
print("="*60)

In [ ]:
# Initialize model
print("\nInitializing model...")
model = GPT2LMHeadModel(model_config)
model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n✓ Model initialized successfully")
print(f"  Total Parameters: {total_params:,}")
print(f"  Trainable Parameters: {trainable_params:,}")
print(f"  Model Size (FP32): {total_params * 4 / 1e6:.2f} MB")
print(f"  Model Size (FP16): {total_params * 2 / 1e6:.2f} MB")

## 7. Data Loading and Preparation

In [ ]:
# Load tokenizer
print("\nLoading tokenizer...")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
print(f"✓ Tokenizer loaded (vocab size: {len(tokenizer):,})")

In [ ]:
# Load dataset
print("\nLoading dataset...")
try:
    dataset = load_dataset(CONFIG['dataset_name'], CONFIG['dataset_config'])
    print(f"✓ Dataset loaded successfully")
    print(f"  Train samples: {len(dataset['train']):,}")
    print(f"  Validation samples: {len(dataset['validation']):,}")
    print(f"  Test samples: {len(dataset['test']):,}")
except Exception as e:
    print(f"Error loading dataset: {e}")
    raise

In [ ]:
# Tokenization function
def tokenize_function(examples):
    """Tokenize text examples"""
    return tokenizer(
        examples['text'],
        truncation=True,
        max_length=CONFIG['max_length'],
        padding='max_length',
        return_tensors='pt'
    )

print("\nTokenizing datasets...")
print("  This may take a few minutes...")

try:
    tokenized_train = dataset['train'].map(
        tokenize_function,
        batched=True,
        remove_columns=dataset['train'].column_names,
        desc="Tokenizing train set"
    )
    
    tokenized_val = dataset['validation'].map(
        tokenize_function,
        batched=True,
        remove_columns=dataset['validation'].column_names,
        desc="Tokenizing validation set"
    )
    
    tokenized_train.set_format('torch')
    tokenized_val.set_format('torch')
    
    print("✓ Tokenization complete")
except Exception as e:
    print(f"Error during tokenization: {e}")
    raise

In [ ]:
# Create dataloaders
print("\nCreating dataloaders...")
try:
    train_loader = DataLoader(
        tokenized_train,
        batch_size=CONFIG['batch_size'],
        shuffle=True,
        num_workers=2,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    val_loader = DataLoader(
        tokenized_val,
        batch_size=CONFIG['batch_size'],
        num_workers=2,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    print(f"✓ DataLoaders created")
    print(f"  Train batches: {len(train_loader):,}")
    print(f"  Validation batches: {len(val_loader):,}")
except Exception as e:
    print(f"Error creating dataloaders: {e}")
    raise

## 8. Optimizer and Scheduler Setup

In [ ]:
# Initialize optimizer
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=CONFIG['learning_rate'],
    weight_decay=CONFIG['weight_decay'],
    betas=(0.9, 0.999),
    eps=1e-8
)

# Calculate total training steps
total_steps = (len(train_loader) * CONFIG['epochs']) // CONFIG['gradient_accumulation_steps']

# Initialize learning rate scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=CONFIG['warmup_steps'],
    num_training_steps=total_steps
)

print("\n" + "="*60)
print("TRAINING SETUP")
print("="*60)
print(f"Total Training Steps: {total_steps:,}")
print(f"Warmup Steps: {CONFIG['warmup_steps']:,}")
print(f"Initial Learning Rate: {CONFIG['learning_rate']}")
print(f"Weight Decay: {CONFIG['weight_decay']}")
print("="*60)

## 9. Resume from Checkpoint (Optional)

In [ ]:
# Resume training from checkpoint if specified
start_epoch = 1
global_step = 0
best_val_loss = float('inf')

if CONFIG['resume_from_checkpoint'] and os.path.exists(CONFIG['resume_from_checkpoint']):
    print(f"\nResuming from checkpoint: {CONFIG['resume_from_checkpoint']}")
    metadata = load_checkpoint(
        CONFIG['resume_from_checkpoint'],
        model,
        optimizer,
        scheduler
    )
    
    if metadata:
        start_epoch = metadata['epoch'] + 1
        global_step = metadata['step']
        if metadata['val_loss']:
            best_val_loss = metadata['val_loss']
        print(f"✓ Resumed from epoch {metadata['epoch']}, step {global_step}")
        print(f"  Previous train loss: {metadata['train_loss']:.4f}")
        if metadata['val_loss']:
            print(f"  Previous val loss: {metadata['val_loss']:.4f}")
else:
    print("\n✓ Starting training from scratch")

## 10. Training Functions

In [ ]:
def train_epoch(model, loader, optimizer, scheduler, device, epoch, config):
    """Train for one epoch with gradient accumulation"""
    model.train()
    total_loss = 0
    optimizer.zero_grad()
    
    progress_bar = tqdm(loader, desc=f"Epoch {epoch}")
    
    for step, batch in enumerate(progress_bar):
        try:
            # Move batch to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            # Forward pass
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=input_ids
            )
            
            # Scale loss for gradient accumulation
            loss = outputs.loss / config['gradient_accumulation_steps']
            loss.backward()
            
            # Update weights after accumulation
            if (step + 1) % config['gradient_accumulation_steps'] == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), config['max_grad_norm'])
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
            
            # Track loss
            total_loss += loss.item() * config['gradient_accumulation_steps']
            
            # Update progress bar
            progress_bar.set_postfix({
                'loss': f"{loss.item() * config['gradient_accumulation_steps']:.4f}",
                'lr': f"{scheduler.get_last_lr()[0]:.2e}"
            })
            
            # Save checkpoint periodically
            if (step + 1) % config['save_steps'] == 0:
                checkpoint_path = os.path.join(
                    config['checkpoint_dir'],
                    f"checkpoint_epoch{epoch}_step{step+1}.pt"
                )
                
                if save_checkpoint(
                    checkpoint_path,
                    model,
                    optimizer,
                    scheduler,
                    epoch,
                    step + 1,
                    total_loss / (step + 1),
                    config=config
                ):
                    print(f"\n✓ Checkpoint saved: {os.path.basename(checkpoint_path)}")
                    cleanup_old_checkpoints(config['checkpoint_dir'], config['max_checkpoints'])
        
        except RuntimeError as e:
            if 'out of memory' in str(e):
                print(f"\n⚠️ OOM Error at step {step}. Clearing cache...")
                torch.cuda.empty_cache()
                gc.collect()
                continue
            else:
                raise e
    
    return total_loss / len(loader)


def evaluate(model, loader, device):
    """Evaluate the model on validation set"""
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating"):
            try:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=input_ids
                )
                total_loss += outputs.loss.item()
            
            except RuntimeError as e:
                if 'out of memory' in str(e):
                    print("\n⚠️ OOM during evaluation. Clearing cache...")
                    torch.cuda.empty_cache()
                    gc.collect()
                    continue
                else:
                    raise e
    
    avg_loss = total_loss / len(loader)
    perplexity = math.exp(avg_loss) if avg_loss < 100 else float('inf')
    return avg_loss, perplexity

print("✓ Training functions defined")

## 11. Main Training Loop

In [ ]:
# Training history
training_history = []

print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)
print(f"Training from epoch {start_epoch} to {CONFIG['epochs']}")
print(f"Checkpoints will be saved to: {CONFIG['checkpoint_dir']}")
print(f"Keeping {CONFIG['max_checkpoints']} most recent checkpoints")
print("="*60 + "\n")

try:
    for epoch in range(start_epoch, CONFIG['epochs'] + 1):
        print(f"\n{'='*60}")
        print(f"EPOCH {epoch}/{CONFIG['epochs']}")
        print(f"{'='*60}")
        
        # Train
        train_loss = train_epoch(
            model,
            train_loader,
            optimizer,
            scheduler,
            device,
            epoch,
            CONFIG
        )
        
        # Evaluate
        val_loss, val_perplexity = evaluate(model, val_loader, device)
        
        # Print results
        print(f"\n{'='*60}")
        print(f"Epoch {epoch} Results:")
        print(f"  Train Loss: {train_loss:.4f}")
        print(f"  Val Loss: {val_loss:.4f}")
        print(f"  Val Perplexity: {val_perplexity:.2f}")
        print(f"{'='*60}")
        
        # Save history
        training_history.append({
            'epoch': epoch,
            'train_loss': float(train_loss),
            'val_loss': float(val_loss),
            'val_perplexity': float(val_perplexity),
            'timestamp': datetime.now().isoformat()
        })
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_path = '/kaggle/working/best_model.pt'
            
            if save_checkpoint(
                best_model_path,
                model,
                optimizer,
                scheduler,
                epoch,
                len(train_loader),
                train_loss,
                val_loss,
                CONFIG
            ):
                print(f"\n✓ New best model saved! (val_loss: {val_loss:.4f})")
        
        # Save epoch checkpoint
        epoch_checkpoint_path = os.path.join(
            CONFIG['checkpoint_dir'],
            f"checkpoint_epoch{epoch}_final.pt"
        )
        
        save_checkpoint(
            epoch_checkpoint_path,
            model,
            optimizer,
            scheduler,
            epoch,
            len(train_loader),
            train_loss,
            val_loss,
            CONFIG
        )
        print(f"✓ Epoch {epoch} checkpoint saved")
        
        # Cleanup old checkpoints
        cleanup_old_checkpoints(CONFIG['checkpoint_dir'], CONFIG['max_checkpoints'])
        
        # Clear cache
        torch.cuda.empty_cache()
        gc.collect()
    
    print("\n" + "="*60)
    print("TRAINING COMPLETED SUCCESSFULLY!")
    print(f"Best Validation Loss: {best_val_loss:.4f}")
    print("="*60)

except KeyboardInterrupt:
    print("\n\n⚠️ Training interrupted by user")
    print("Saving emergency checkpoint...")
    emergency_path = '/kaggle/working/emergency_checkpoint.pt'
    save_checkpoint(
        emergency_path,
        model,
        optimizer,
        scheduler,
        epoch,
        step,
        train_loss,
        config=CONFIG
    )
    print(f"✓ Emergency checkpoint saved to {emergency_path}")

except Exception as e:
    print(f"\n\n❌ Training failed with error: {e}")
    import traceback
    traceback.print_exc()
    raise

## 12. Save Training History

In [ ]:
# Save training history
history_path = '/kaggle/working/training_history.json'
with open(history_path, 'w') as f:
    json.dump(training_history, f, indent=2)

print("\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)
for entry in training_history:
    print(f"Epoch {entry['epoch']}: "
          f"Train Loss={entry['train_loss']:.4f}, "
          f"Val Loss={entry['val_loss']:.4f}, "
          f"Perplexity={entry['val_perplexity']:.2f}")
print("="*60)
print(f"\n✓ Training history saved to {history_path}")

## 13. Text Generation Test

In [ ]:
def generate_text(prompt, max_length=100, temperature=0.8, num_return_sequences=1):
    """Generate text from a prompt"""
    model.eval()
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            temperature=temperature,
            top_k=50,
            top_p=0.95,
            do_sample=True,
            num_return_sequences=num_return_sequences,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    return [tokenizer.decode(seq, skip_special_tokens=True) for seq in output]

# Test prompts
test_prompts = [
    "The future of artificial intelligence",
    "In a world where technology",
    "Scientists have discovered",
    "Once upon a time in a distant land"
]

print("\n" + "="*60)
print("TEXT GENERATION EXAMPLES")
print("="*60)

for prompt in test_prompts:
    print(f"\n{'─'*60}")
    print(f"Prompt: '{prompt}'")
    print(f"{'─'*60}")
    try:
        generated = generate_text(prompt, max_length=150, temperature=0.8)
        print(generated[0])
    except Exception as e:
        print(f"Error generating text: {e}")
    print()

## 14. Save Final Model

In [ ]:
# Save model in HuggingFace format
final_model_dir = '/kaggle/working/final_model'
print(f"\nSaving final model to {final_model_dir}...")

try:
    model.save_pretrained(final_model_dir)
    tokenizer.save_pretrained(final_model_dir)
    
    # Save training config
    config_path = os.path.join(final_model_dir, 'training_config.json')
    with open(config_path, 'w') as f:
        json.dump(CONFIG, f, indent=2)
    
    print("✓ Model saved successfully")
except Exception as e:
    print(f"Error saving model: {e}")

## 15. Output Summary

In [ ]:
# List all output files
print("\n" + "="*60)
print("OUTPUT FILES")
print("="*60)
print("\n📁 /kaggle/working/")
print("  ├── best_model.pt (best checkpoint based on validation loss)")
print("  ├── training_history.json (training metrics)")
print("  ├── checkpoints/ (training checkpoints)")

# List checkpoints
checkpoints = get_checkpoint_list(CONFIG['checkpoint_dir'])
if checkpoints:
    for ckpt in checkpoints:
        size_mb = os.path.getsize(ckpt) / 1e6
        print(f"  │   ├── {os.path.basename(ckpt)} ({size_mb:.1f} MB)")

print("  └── final_model/ (HuggingFace format)")
print("      ├── pytorch_model.bin")
print("      ├── config.json")
print("      ├── training_config.json")
print("      └── tokenizer files")
print("="*60)

# Calculate total size
total_size = 0
for root, dirs, files in os.walk('/kaggle/working'):
    for file in files:
        filepath = os.path.join(root, file)
        total_size += os.path.getsize(filepath)

print(f"\nTotal output size: {total_size / 1e6:.2f} MB")
print("\n✓ Training complete! Download the files from the Output tab.")